In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import StratifiedKFold
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import auc
from sklearn import metrics

/opt/python/python35/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/opt/python/python35/lib/python3.5/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


## Data Prep

In [ ]:
path = '/axp/rim/imsads/dev/sagra39/IMS_Hackathon_17/input/'
veid_data_sample = pd.read_csv(path + 'veid_data.csv')
Browsing_History_Sample = pd.read_csv(path + 'Browsing_History.csv')
DEVICE_DATA_Sample = pd.read_csv(path + 'device_data.csv', encoding='iso-8859-1' )
IP3_APP_sample = pd.read_csv(path + 'IP3_APP.csv')
IP3_CM_sample = pd.read_csv(path + 'IP3_CM.csv')
modelling_sample = pd.read_csv(path + 'Modeling.csv')

print (DEVICE_DATA_Sample.shape, IP3_APP_sample.shape, IP3_CM_sample.shape, modelling_sample.shape)

In [ ]:
#import datetime
modelling_sample['hour'] = modelling_sample['datetime'].map(lambda x : datetime.datetime.strptime(x, '%d%b%Y:%H:%M:%S').hour )
modelling_sample['day'] = modelling_sample['datetime'].map(lambda x : datetime.datetime.strptime(x, '%d%b%Y:%H:%M:%S').day )
modelling_sample['month'] = modelling_sample['datetime'].map(lambda x : datetime.datetime.strptime(x, '%d%b%Y:%H:%M:%S').month )
modelling_sample['weekday'] = modelling_sample['datetime'].map(lambda x : datetime.datetime.strptime(x, '%d%b%Y:%H:%M:%S').weekday() )


In [ ]:
modelling_IP3CM_APP = modelling_sample.merge(IP3_CM_sample, on='ip_3', how = 'left').merge(IP3_APP_sample, on='ip_3', how = 'left')
modelling_IP3CM_APP = modelling_IP3CM_APP.fillna(-999)
print (modelling_IP3CM_APP.shape, modelling_sample.shape, IP3_CM_sample.shape, IP3_APP_sample.shape)

In [ ]:
DEVICE_DATA_Sample = DEVICE_DATA_Sample.rename(columns={'UserAgent':"user_agent"})
modelling_IP3CM_APP_device = modelling_IP3CM_APP.merge(DEVICE_DATA_Sample, on='user_agent', how = 'left')

## Browsing Attributes

In [ ]:
modelling_IP3CM_APP_device['new_var1'] = modelling_IP3CM_APP_device['Product_Color_Desc'].map(lambda x: 1 if x == 'GOLD' else 0)
modelling_IP3CM_APP_device['new_var2'] = modelling_IP3CM_APP_device['Product_Color_Desc'].map(lambda x: 1 if x == 'PLATINUM' else 0)
modelling_IP3CM_APP_device['new_var3'] = modelling_IP3CM_APP_device['sub_channel'].map(lambda x: 1 if x == 'Interactive' else 0)

modelling_IP3CM_APP_device['new_var4'] = modelling_IP3CM_APP_device['product_list'].map(lambda x: 1 if x == ';UK:ICSS:E77' else 0)
modelling_IP3CM_APP_device['new_var5'] = modelling_IP3CM_APP_device['product_list'].map(lambda x: 1 if x == ';UK:ICSS:E82' else 0)
modelling_IP3CM_APP_device['new_var6'] = modelling_IP3CM_APP_device['product_list'].map(lambda x: 1 if x == ';UK:ICSS:I73' else 0)

In [ ]:
browsing_path = '/axp/rim/imsads/dev/sagra39/IMS_Hackathon_17/input/browsing/'
browsing_data_vars = pd.read_csv(browsing_path + 'browsing_data_vars.csv', header=None)
browsing_data_vars1 = pd.read_csv(browsing_path + 'browsing_data_vars1.csv', header=None)
browsing_data_vars2 = pd.read_csv(browsing_path + 'browsing_data_vars2.csv', header=None)
browsing_data_vars3 = pd.read_csv(browsing_path + 'browsing_data_vars3.csv', header=None)
browsing_data_vars4 = pd.read_csv(browsing_path + 'browsing_data_vars4.csv', header=None)
browsing_data_vars5 = pd.read_csv(browsing_path + 'browsing_data_vars5.csv', header=None)
browsing_data_vars6 = pd.read_csv(browsing_path + 'browsing_data_vars6.csv', header=None)
print (browsing_data_vars.shape, browsing_data_vars1.shape, browsing_data_vars2.shape, browsing_data_vars3.shape, browsing_data_vars4.shape, browsing_data_vars5.shape, browsing_data_vars6.shape )

In [ ]:
browsing_data_vars.columns=[['pcn']+[str(i)+'_br' for i in list(range(0,11))]]
browsing_data_vars1.columns=[['pcn']+[str(i)+'_br1' for i in list(range(0,171))]]
browsing_data_vars2.columns=[['pcn']+[str(i)+'_br2' for i in list(range(0,13))]]
browsing_data_vars3.columns=[['pcn']+[str(i)+'_br3' for i in list(range(0,159))]]
browsing_data_vars4.columns=[['pcn']+[str(i)+'_br4' for i in list(range(0,139))]]
browsing_data_vars5.columns=[['pcn']+[str(i)+'_br5' for i in list(range(0,42))]]
browsing_data_vars6.columns=[['pcn']+[str(i)+'_br6' for i in list(range(0,127))]]

In [ ]:
modelling_IP3CM_APP_device_brows = modelling_IP3CM_APP_device.merge(browsing_data_vars, on = 'pcn', how = 'left').merge(browsing_data_vars1, on = 'pcn', how = 'left').merge(browsing_data_vars2,on = 'pcn', how = 'left').merge(browsing_data_vars3, on = 'pcn', how = 'left').merge(browsing_data_vars4, on = 'pcn', how = 'left').merge(browsing_data_vars5,on = 'pcn', how = 'left').merge(browsing_data_vars6, on = 'pcn', how = 'left')

modelling_IP3CM_APP_device_brows.shape

## Train, Valid and Test data split

In [ ]:
## Train, Valid, Test split

import copy
final_data = copy.deepcopy(modelling_IP3CM_APP_device_brows)


pcn  = pd.DataFrame({'pcn':list(set(final_data['pcn']))})
train = pcn.sample(frac=0.7,random_state=1)
valid = set(pcn['pcn'])-set(train['pcn'])
valid = pd.DataFrame({'pcn':list(valid)})
valid = valid.sample(frac=0.67,random_state=1)

test  = set(pcn['pcn'])-set(train['pcn'])-set(valid['pcn'])
test  = pd.DataFrame({'pcn':list(test)})

print(train.shape)
print(valid.shape)
print(test.shape)

print(set(train['pcn'])&set(valid['pcn'])&set(test['pcn']))

train = train.merge(final_data, on=['pcn'], how='left')
valid = valid.merge(final_data, on=['pcn'], how='left')
test  = test.merge(final_data, on=['pcn'], how='left')

print(train.shape)
print(valid.shape)
print(test.shape)

In [ ]:
list_drop = ['pcn', 'post_uni_visid', 'datetime', 'Market_cd_x', 'Market_cd_y']
train_x = train.drop(list_drop, axis=1)
train_y = train['Gamer_flag']

valid_x = valid.drop(list_drop, axis=1)
valid_y = valid['Gamer_flag']

test_x = test.drop(list_drop, axis=1)
test_y = test['Gamer_flag']


list_cat= []
for i in train_x:
    if train_x[i].dtypes == 'object':
        list_cat.append(i)
list_cat

In [ ]:
train_x = train_x.fillna(-999)
valid_x = valid_x.fillna(-999)
test_x = test_x.fillna(-999)
for i in list_cat:
        train_x[i] = train_x[i].map(lambda x : str(x))
        valid_x[i] = valid_x[i].map(lambda x : str(x))
        test_x[i] = test_x[i].map(lambda x : str(x))
        
        le = preprocessing.LabelEncoder()
        train_x[i] = le.fit_transform(train_x[i])
        valid_x[i] = le.fit_transform(valid_x[i])
        test_x[i] = le.fit_transform(test_x[i])

## Final submission data

In [ ]:
path = '/axp/rim/imsads/dev/sagra39/IMS_Hackathon_17/input/'
DEVICE_DATA_Sample = pd.read_csv(path + 'device_data.csv', encoding='iso-8859-1' )
IP3_APP_sample = pd.read_csv(path + 'IP3_APP.csv')
IP3_CM_sample = pd.read_csv(path + 'IP3_CM.csv')
modelling_sample = pd.read_csv(path + 'Modeling.csv')
print (DEVICE_DATA_Sample.shape, IP3_APP_sample.shape, IP3_CM_sample.shape, modelling_sample.shape) 

In [ ]:
path = '/axp/rim/imsads/dev/sagra39/IMS_Hackathon_17/input/'
InterimEvaluation = pd.read_csv(path + 'FinalEvaluation.csv')
print (InterimEvaluation.shape, IP3_APP_sample.shape, IP3_CM_sample.shape, modelling_sample.shape)

In [ ]:
import datetime
InterimEvaluation['hour'] = InterimEvaluation['datetime'].map(lambda x : datetime.datetime.strptime(x, '%d%b%Y:%H:%M:%S').hour )
InterimEvaluation['day'] = InterimEvaluation['datetime'].map(lambda x : datetime.datetime.strptime(x, '%d%b%Y:%H:%M:%S').day )
InterimEvaluation['month'] = InterimEvaluation['datetime'].map(lambda x : datetime.datetime.strptime(x, '%d%b%Y:%H:%M:%S').month )
InterimEvaluation['weekday'] = InterimEvaluation['datetime'].map(lambda x : datetime.datetime.strptime(x, '%d%b%Y:%H:%M:%S').weekday() )

In [ ]:
InterimEvaluation_IP3CM_APP = InterimEvaluation.merge(IP3_CM_sample, on='ip_3', how = 'left').merge(IP3_APP_sample, on='ip_3', how = 'left')
InterimEvaluation_IP3CM_APP = InterimEvaluation_IP3CM_APP.fillna(-999)
print (InterimEvaluation_IP3CM_APP.shape, modelling_sample.shape, IP3_CM_sample.shape, IP3_APP_sample.shape) 

In [ ]:
DEVICE_DATA_Sample = DEVICE_DATA_Sample.rename(columns={'UserAgent':"user_agent"})
InterimEvaluation_IP3CM_APP_device = InterimEvaluation_IP3CM_APP.merge(DEVICE_DATA_Sample, on='user_agent', how = 'left') 

In [ ]:
InterimEvaluation_IP3CM_APP_device['new_var1'] = InterimEvaluation_IP3CM_APP_device['Product_Color_Desc'].map(lambda x: 1 if x == 'GOLD' else 0)
InterimEvaluation_IP3CM_APP_device['new_var2'] = InterimEvaluation_IP3CM_APP_device['Product_Color_Desc'].map(lambda x: 1 if x == 'PLATINUM' else 0)
InterimEvaluation_IP3CM_APP_device['new_var3'] = InterimEvaluation_IP3CM_APP_device['sub_channel'].map(lambda x: 1 if x == 'Interactive' else 0)

InterimEvaluation_IP3CM_APP_device['new_var4'] = InterimEvaluation_IP3CM_APP_device['product_list'].map(lambda x: 1 if x == ';UK:ICSS:E77' else 0)
InterimEvaluation_IP3CM_APP_device['new_var5'] = InterimEvaluation_IP3CM_APP_device['product_list'].map(lambda x: 1 if x == ';UK:ICSS:E82' else 0)
InterimEvaluation_IP3CM_APP_device['new_var6'] = InterimEvaluation_IP3CM_APP_device['product_list'].map(lambda x: 1 if x == ';UK:ICSS:I73' else 0)

In [ ]:
InterimEvaluation_IP3CM_APP_device_brows = InterimEvaluation_IP3CM_APP_device.merge(browsing_data_vars, on = 'pcn', how = 'left').merge(browsing_data_vars1, on = 'pcn', how = 'left').merge(browsing_data_vars2,on = 'pcn', how = 'left').merge(browsing_data_vars3, on = 'pcn', how = 'left').merge(browsing_data_vars4, on = 'pcn', how = 'left').merge(browsing_data_vars5,on = 'pcn', how = 'left').merge(browsing_data_vars6, on = 'pcn', how = 'left')

InterimEvaluation_IP3CM_APP_device_brows.shape

In [ ]:
list_drop = ['pcn', 'post_uni_visid', 'datetime', 'Market_cd_x', 'Market_cd_y']
final_sub = InterimEvaluation_IP3CM_APP_device_brows.drop(list_drop, axis=1)
#final_sub_y = InterimEvaluation_IP3CM_APP_device_brows['Gamer_flag']

list_cat=['ip','geo_region', 'geo_city',
 'product_list',
 'user_agent',
 'Product_Color_Desc',
 'sub_channel',
 'ip_3',
 'DeviceType',
 'Osname',
 'Browsername',
 'primaryHardwareType',
 'model',
 '8_br',
 '9_br']

In [ ]:
for i in list_cat:
        print(i)
        final_sub[i] = final_sub[i].map(lambda x : str(x))
        
        le = preprocessing.LabelEncoder()

        final_sub[i] = le.fit_transform(final_sub[i])

## LightGBM model

In [ ]:
model=lgb.LGBMClassifier(boosting_type='gbdt', num_leaves=35, max_depth=-1, learning_rate=0.005, n_estimators=1500, max_bin=255, subsample_for_bin=50000, objective=None, min_split_gain=0.0, min_child_weight=5, min_child_samples=10, subsample=1.0, subsample_freq=1, colsample_bytree=1.0, reg_alpha=0.0, reg_lambda=0.0, silent=True)

In [ ]:
model.fit(train_x,train_y,
        eval_set=[(valid_x,valid_y)],
        eval_metric='logloss',
        early_stopping_rounds=30) 

In [ ]:
pred_valid =model.predict_proba(valid_x)
pred_test =model.predict_proba(test_x)
pred_train =model.predict_proba(train_x)

In [ ]:
dt_valid=pd.DataFrame(pred_valid)
dt_test=pd.DataFrame(pred_test)
dt_train=pd.DataFrame(pred_train)

dt_valid.columns=['0_score','1_score']
dt_test.columns=['0_score','1_score']
dt_train.columns=['0_score','1_score']
print(dt_valid.shape, dt_test.shape, dt_train.shape)

In [ ]:
# Evaluation
from sklearn.metrics import roc_curve, auc
fpr_valid,tpr_valid,thresholds_valid = roc_curve(valid_y,dt_valid['1_score'])
fpr_test,tpr_test,thresholds_test = roc_curve(test_y,dt_test['1_score'])
fpr_train,tpr_train,thresholds_train = roc_curve(train_y,dt_train['1_score'])

print ('valid:', 2*auc(fpr_valid,tpr_valid) - 1, 'test:', 2*auc(fpr_test,tpr_test) - 1,   'train:', 2*auc(fpr_train,tpr_train) - 1)

## Prediction on Final data 

In [ ]:
pred_valid = pd.DataFrame(model.predict_proba(final_sub), columns=['score'])
dt_final_sub_pred=pd.DataFrame(final_sub_pred)
dt_final_sub_pred.columns=['0_score','1_score']

In [ ]:
dt_final_sub_pred['gamer_flag'] = dt_final_sub_pred['1_score'].map(lambda x : 1 if x>0.176 else 0)

In [ ]:
x=InterimEvaluation[['pcn']].join(dt_final_sub_pred['gamer_flag'])

In [ ]:
x.to_csv('/axp/rim/imsads/dev/sagra39/IMS_Hackathon_17/input/GGN23_Final.csv',index=False)